<a href="https://colab.research.google.com/github/shashi3876/kaggle/blob/main/PodcastListenPredictLightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

THis notebook contains the work done for the Kaggel playgorund competition named Predict Podcast Listening Time. Mainly I'm using LIghtGBM in this notebook for the predcitions.

In [19]:
# imports

import pandas as pd

In [31]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [32]:
train.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


# Preprocess Data

First I'm filling the null values of train and test set.

In [33]:
train.isna().sum()

,0
id,0
Podcast_Name,0
Episode_Title,0
Episode_Length_minutes,87093
Genre,0
Host_Popularity_percentage,0
Publication_Day,0
Publication_Time,0
Guest_Popularity_percentage,146030
Number_of_Ads,1


In [34]:
train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
test['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)

train['Number_of_Ads'].fillna(train['Number_of_Ads'].median(), inplace=True)
train['Guest_Popularity_percentage'].fillna(train['Guest_Popularity_percentage'].median(), inplace=True)


test['Guest_Popularity_percentage'].fillna(test['Guest_Popularity_percentage'].median(), inplace=True)

<ipython-input-34-099d9f371e57>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Episode_Length_minutes'].fillna(train['Episode_Length_minutes'].median(), inplace=True)
<ipython-input-34-099d9f371e57>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

Now doing the label encoding on the categorical data

In [35]:
categorical_features = [
    'Podcast_Name',
    'Genre',
    'Publication_Day',
    'Publication_Time',
    'Episode_Sentiment'
]
from sklearn.preprocessing import LabelEncoder

# Encoder for categorical data
label_encoders = {col: LabelEncoder() for col in categorical_features}

# Apply LabelEncoder to each categorical column
for col in categorical_features:
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

    # Converting to category type
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [36]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

In [37]:
train.head()

,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num
0,34,63.84,9,74.81,4,3,53.58,0.0,2,31.41998,98
1,24,119.80,1,66.95,2,0,75.95,2.0,0,88.01241,26
2,40,73.90,2,69.97,5,1,8.97,0.0,0,44.92531,16
3,10,67.17,8,57.22,1,2,78.70,2.0,2,46.27824,45
4,31,110.51,3,80.07,1,0,58.68,3.0,1,75.61031,86


In [38]:
train.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num'],
      dtype='object')

In [39]:
categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']

# Build the model

In [40]:
X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

In [41]:
X.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Episode_Num'],
      dtype='object')

In [42]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)

<ipython-input-42-01d91ff83fe4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-42-01d91ff83fe4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2593
[200]	valid_0's rmse: 12.9037
[300]	valid_0's rmse: 12.8683
[400]	valid_0's rmse: 12.8551
[500]	valid_0's rmse: 12.8429
[600]	valid_0's rmse: 12.8343
[700]	valid_0's rmse: 12.8272
[800]	valid_0's rmse: 12.8204
[900]	valid_0's rmse: 12.813
[1000]	valid_0's rmse: 12.8083
[1100]	valid_0's rmse: 12.8051
[1200]	valid_0's rmse: 12.8002
[1300]	valid_0's rmse: 12.796
[1400]	valid_0's rmse: 12.7917
[1500]	valid_0's rmse: 12.788
[1600]	valid_0's rmse: 12.7856
[1700]	valid_0's rmse: 12.7817
[1800]	valid_0's rmse: 12.7794
[1900]	valid_0's rmse: 12.7768
[2000]	valid_0's rmse: 12.7753
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 12.7753


<ipython-input-42-01d91ff83fe4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-42-01d91ff83fe4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3112
[200]	valid_0's rmse: 12.9562
[300]	valid_0's rmse: 12.9238
[400]	valid_0's rmse: 12.9084
[500]	valid_0's rmse: 12.8962
[600]	valid_0's rmse: 12.8885
[700]	valid_0's rmse: 12.881
[800]	valid_0's rmse: 12.8728
[900]	valid_0's rmse: 12.8677
[1000]	valid_0's rmse: 12.8629
[1100]	valid_0's rmse: 12.8603
[1200]	valid_0's rmse: 12.8581
[1300]	valid_0's rmse: 12.8551
[1400]	valid_0's rmse: 12.8528
[1500]	valid_0's rmse: 12.8496
[1600]	valid_0's rmse: 12.8482
[1700]	valid_0's rmse: 12.8445
[1800]	valid_0's rmse: 12.8432
[1900]	valid_0's rmse: 12.8412
[2000]	valid_0's rmse: 12.8395
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 12.8395


<ipython-input-42-01d91ff83fe4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-42-01d91ff83fe4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.3036
[200]	valid_0's rmse: 12.9452
[300]	valid_0's rmse: 12.9109
[400]	valid_0's rmse: 12.8943
[500]	valid_0's rmse: 12.8832
[600]	valid_0's rmse: 12.8737
[700]	valid_0's rmse: 12.8671
[800]	valid_0's rmse: 12.8595
[900]	valid_0's rmse: 12.8547
[1000]	valid_0's rmse: 12.848
[1100]	valid_0's rmse: 12.8448
[1200]	valid_0's rmse: 12.8389
[1300]	valid_0's rmse: 12.8339
[1400]	valid_0's rmse: 12.8305
[1500]	valid_0's rmse: 12.8283
[1600]	valid_0's rmse: 12.8262
[1700]	valid_0's rmse: 12.8235
[1800]	valid_0's rmse: 12.8207
[1900]	valid_0's rmse: 12.8198
[2000]	valid_0's rmse: 12.8172
Did not meet early stopping. Best iteration is:
[1949]	valid_0's rmse: 12.8172


<ipython-input-42-01d91ff83fe4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-42-01d91ff83fe4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.319
[200]	valid_0's rmse: 12.9564
[300]	valid_0's rmse: 12.9226
[400]	valid_0's rmse: 12.9065
[500]	valid_0's rmse: 12.8937
[600]	valid_0's rmse: 12.8866
[700]	valid_0's rmse: 12.8762
[800]	valid_0's rmse: 12.8676
[900]	valid_0's rmse: 12.8628
[1000]	valid_0's rmse: 12.8605
[1100]	valid_0's rmse: 12.8572
[1200]	valid_0's rmse: 12.8522
[1300]	valid_0's rmse: 12.8471
[1400]	valid_0's rmse: 12.8434
[1500]	valid_0's rmse: 12.8427
[1600]	valid_0's rmse: 12.8392
[1700]	valid_0's rmse: 12.8384
[1800]	valid_0's rmse: 12.837
[1900]	valid_0's rmse: 12.834
[2000]	valid_0's rmse: 12.8334
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 12.8334


<ipython-input-42-01d91ff83fe4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-42-01d91ff83fe4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2473
[200]	valid_0's rmse: 12.9025
[300]	valid_0's rmse: 12.8715
[400]	valid_0's rmse: 12.857
[500]	valid_0's rmse: 12.8451
[600]	valid_0's rmse: 12.8369
[700]	valid_0's rmse: 12.8262
[800]	valid_0's rmse: 12.8202
[900]	valid_0's rmse: 12.8133
[1000]	valid_0's rmse: 12.8065
[1100]	valid_0's rmse: 12.8019
[1200]	valid_0's rmse: 12.7943
[1300]	valid_0's rmse: 12.7897
[1400]	valid_0's rmse: 12.786
[1500]	valid_0's rmse: 12.7823
[1600]	valid_0's rmse: 12.7811
[1700]	valid_0's rmse: 12.7786
[1800]	valid_0's rmse: 12.7764
[1900]	valid_0's rmse: 12.7736
[2000]	valid_0's rmse: 12.772
Did not meet early stopping. Best iteration is:
[1963]	valid_0's rmse: 12.7718


In [ ]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_2.csv', index=False)

# Fill the missing values using group wise impuation

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

# Fill train missing values by mean per Podcast_Name group
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].fillna(
    train.groupby('Podcast_Name')['Episode_Length_minutes'].transform('mean')
)

train['Number_of_Ads'] = train['Number_of_Ads'].fillna(
    train.groupby('Podcast_Name')['Number_of_Ads'].transform('mean')
)

train['Guest_Popularity_percentage'] = train['Guest_Popularity_percentage'].fillna(
    train.groupby('Podcast_Name')['Guest_Popularity_percentage'].transform('mean')
)

# For test set: use group means from the train set (safe approach)
podcast_means = train.groupby('Podcast_Name')[['Episode_Length_minutes', 'Guest_Popularity_percentage']].mean()

# Map those means to the test set
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].fillna(
    test['Podcast_Name'].map(podcast_means['Episode_Length_minutes'])
)

test['Guest_Popularity_percentage'] = test['Guest_Popularity_percentage'].fillna(
    test['Podcast_Name'].map(podcast_means['Guest_Popularity_percentage'])

)

In [11]:
train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title','id'])
test = test.drop(columns=['Episode_Title','id'])

X = train.drop(columns=['Listening_Time_minutes'])
y = train['Listening_Time_minutes']

categorical_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time','Episode_Sentiment', 'Episode_Num']
cv = KFold(5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoded_columns = categorical_cols
    encoder = TargetEncoder(random_state=42)

    X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
    X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
    X_test[encoded_columns] = encoder.transform(X_test[encoded_columns])


    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
            ],
    )

    y_pred += model.predict(X_test)


<ipython-input-11-2bdc70af3650>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-11-2bdc70af3650>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2141
[200]	valid_0's rmse: 12.9472
[300]	valid_0's rmse: 12.924
[400]	valid_0's rmse: 12.9143
[500]	valid_0's rmse: 12.9093
[600]	valid_0's rmse: 12.9033
[700]	valid_0's rmse: 12.8989
[800]	valid_0's rmse: 12.8961
[900]	valid_0's rmse: 12.8912
[1000]	valid_0's rmse: 12.8895
[1100]	valid_0's rmse: 12.8863
[1200]	valid_0's rmse: 12.8854
[1300]	valid_0's rmse: 12.8841
[1400]	valid_0's rmse: 12.8819
[1500]	valid_0's rmse: 12.8809
[1600]	valid_0's rmse: 12.8787
Early stopping, best iteration is:
[1558]	valid_0's rmse: 12.8785


<ipython-input-11-2bdc70af3650>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-11-2bdc70af3650>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2683
[200]	valid_0's rmse: 13.0022
[300]	valid_0's rmse: 12.9815
[400]	valid_0's rmse: 12.9719
[500]	valid_0's rmse: 12.9679
[600]	valid_0's rmse: 12.9619
[700]	valid_0's rmse: 12.9607
[800]	valid_0's rmse: 12.9573
[900]	valid_0's rmse: 12.9546
[1000]	valid_0's rmse: 12.9521
[1100]	valid_0's rmse: 12.9482
[1200]	valid_0's rmse: 12.9475
[1300]	valid_0's rmse: 12.9467
[1400]	valid_0's rmse: 12.9454
[1500]	valid_0's rmse: 12.9445
[1600]	valid_0's rmse: 12.942
[1700]	valid_0's rmse: 12.9416
[1800]	valid_0's rmse: 12.9408
[1900]	valid_0's rmse: 12.9402
[2000]	valid_0's rmse: 12.9386
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 12.9386


<ipython-input-11-2bdc70af3650>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-11-2bdc70af3650>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.265
[200]	valid_0's rmse: 12.9999
[300]	valid_0's rmse: 12.9736
[400]	valid_0's rmse: 12.9635
[500]	valid_0's rmse: 12.9584
[600]	valid_0's rmse: 12.9548
[700]	valid_0's rmse: 12.9499
[800]	valid_0's rmse: 12.9464
[900]	valid_0's rmse: 12.9423
[1000]	valid_0's rmse: 12.9417
[1100]	valid_0's rmse: 12.9392
[1200]	valid_0's rmse: 12.9378
[1300]	valid_0's rmse: 12.9358
[1400]	valid_0's rmse: 12.9359
Early stopping, best iteration is:
[1376]	valid_0's rmse: 12.9353


<ipython-input-11-2bdc70af3650>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-11-2bdc70af3650>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2686
[200]	valid_0's rmse: 12.9939
[300]	valid_0's rmse: 12.9684
[400]	valid_0's rmse: 12.9543
[500]	valid_0's rmse: 12.9479
[600]	valid_0's rmse: 12.943
[700]	valid_0's rmse: 12.9392
[800]	valid_0's rmse: 12.9352
[900]	valid_0's rmse: 12.9314
[1000]	valid_0's rmse: 12.9293
[1100]	valid_0's rmse: 12.9274
[1200]	valid_0's rmse: 12.9245
[1300]	valid_0's rmse: 12.9226
[1400]	valid_0's rmse: 12.9191
[1500]	valid_0's rmse: 12.9183
[1600]	valid_0's rmse: 12.9181
Early stopping, best iteration is:
[1518]	valid_0's rmse: 12.9173


<ipython-input-11-2bdc70af3650>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[encoded_columns] = encoder.fit_transform(X_train[encoded_columns], y_train)
<ipython-input-11-2bdc70af3650>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[encoded_columns] = encoder.transform(X_valid[encoded_columns])
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it in

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 13.2137
[200]	valid_0's rmse: 12.9538
[300]	valid_0's rmse: 12.9319
[400]	valid_0's rmse: 12.9172
[500]	valid_0's rmse: 12.9121
[600]	valid_0's rmse: 12.9063
[700]	valid_0's rmse: 12.9043
[800]	valid_0's rmse: 12.9007
[900]	valid_0's rmse: 12.8971
[1000]	valid_0's rmse: 12.8934
[1100]	valid_0's rmse: 12.8909
[1200]	valid_0's rmse: 12.8895
[1300]	valid_0's rmse: 12.8866
[1400]	valid_0's rmse: 12.8847
[1500]	valid_0's rmse: 12.8854
Early stopping, best iteration is:
[1407]	valid_0's rmse: 12.8845


In [43]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Listening_Time_minutes' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm_3.csv', index=False)

# Add combined categorical features

In [16]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

train['Episode_Num'] = train['Episode_Title'].str[8:].astype('category')
test['Episode_Num'] = test['Episode_Title'].str[8:].astype('category')

train = train.drop(columns=['Episode_Title'])
test = test.drop(columns=['Episode_Title'])

# Fill train missing values by mean per Podcast_Name group
train['Episode_Length_minutes'] = train['Episode_Length_minutes'].fillna(
    train.groupby('Podcast_Name')['Episode_Length_minutes'].transform('mean')
)

train['Number_of_Ads'] = train['Number_of_Ads'].fillna(
    train.groupby('Podcast_Name')['Number_of_Ads'].transform('mean')
)

train['Guest_Popularity_percentage'] = train['Guest_Popularity_percentage'].fillna(
    train.groupby('Podcast_Name')['Guest_Popularity_percentage'].transform('mean')
)

# For test set: use group means from the train set (safe approach)
podcast_means = train.groupby('Podcast_Name')[['Episode_Length_minutes', 'Guest_Popularity_percentage']].mean()

# Map those means to the test set
test['Episode_Length_minutes'] = test['Episode_Length_minutes'].fillna(
    test['Podcast_Name'].map(podcast_means['Episode_Length_minutes'])
)

test['Guest_Popularity_percentage'] = test['Guest_Popularity_percentage'].fillna(
    test['Podcast_Name'].map(podcast_means['Guest_Popularity_percentage'])

)



In [17]:
from tqdm import tqdm
cat_cols = [
    'Podcast_Name', 'Genre', 'Publication_Day',
    'Publication_Time', 'Episode_Sentiment', 'Episode_Num'
]


combined_features =  cat_cols

# Create combinations of 2 features (can go 2, 3, 4 but heavy)
from itertools import combinations
pair_size = [2]

for r in pair_size:
    combos = list(combinations(combined_features, r))
    for cols in tqdm(combos):
        col_name = '_'.join(cols)
        train[col_name] = train[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')
        test[col_name] = test[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')

100%|██████████| 15/15 [01:14<00:00,  4.96s/it]


In [18]:
train.columns

Index(['id', 'Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num', 'Podcast_Name_Genre',
       'Podcast_Name_Publication_Day', 'Podcast_Name_Publication_Time',
       'Podcast_Name_Episode_Sentiment', 'Podcast_Name_Episode_Num',
       'Genre_Publication_Day', 'Genre_Publication_Time',
       'Genre_Episode_Sentiment', 'Genre_Episode_Num',
       'Publication_Day_Publication_Time', 'Publication_Day_Episode_Sentiment',
       'Publication_Day_Episode_Num', 'Publication_Time_Episode_Sentiment',
       'Publication_Time_Episode_Num', 'Episode_Sentiment_Episode_Num'],
      dtype='object')